In [ ]:
publishers_type = 'Journalist' 
input_file_name = './kpp.csv'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import tldextract
import numpy as np
from d3graph import d3graph, vec2adjmat

In [ ]:
scope_df = pd.read_csv('./citation_scope.csv')
node_list = list(scope_df.loc[scope_df['Tag: type'] == publishers_type].Name.values)
node_list.sort()
node_list = list(set(node_list))
node_list

In [ ]:
crawl_scope = pd.read_csv('./crawl_scope.csv')
crawl_scope.fillna('', inplace=True)
crawl_list1 = list(set(crawl_scope.loc[crawl_scope['Tag: type'] == publishers_type]["Twitter Handle"].values))
crawl_list2 = list(set(crawl_scope.loc[crawl_scope['Tag: type'] == publishers_type]["Name"].values))
crawl_list = crawl_list1 + crawl_list2
crawl_list

In [ ]:
node_list = node_list + crawl_list
for i in range(len(node_list)):
    node_list[i] = node_list[i].strip()
node_list

In [ ]:
# read the postprocessed data file (.csv) as a pandas dataframe
with open(input_file_name, encoding='utf-8', newline='') as f:
    df_raw = pd.read_csv(input_file_name)

# set the IDs as indices
df_raw.set_index('id', inplace=True)

In [ ]:
'Amnesty International' in node_list
'@hrw' in node_list

In [ ]:
df_raw.head()

In [ ]:
def find_referral_relation(df, node_list):
    referral_relation = {}
    for index, row in df.iterrows():
        citation_names = ast.literal_eval(row['citation name'])
        if row["type"] != "twitter":
                domain = tldextract.extract(row["url"])[1]
        else:
            domain = "@" + row["url"].split("/")[3]
        for citation_name in citation_names:
            if citation_name.strip() in node_list and domain.strip() in node_list:
                if citation_name != domain:
                    if (domain, citation_name) in referral_relation:
                        referral_relation[(domain, citation_name)] += 1
                    else:
                        referral_relation[(domain, citation_name)] = 1
                
    return referral_relation

In [ ]:
find_referral_relation(df_raw.head(), node_list)

In [ ]:
referral_relation = find_referral_relation(df_raw, node_list)
referral_relation

In [ ]:
source = []
target = []
weight = []
for key, value in referral_relation.items():
    source.append(key[0])
    target.append(key[1])
    weight.append(value)

adjmat = vec2adjmat(source, target, weight=weight)
adjmat

In [ ]:
d3 = d3graph()
d3.graph(adjmat)
# size = [10,20,10,10,15,10,5]
d3.set_edge_properties(directed=True)
# d3.set_node_properties(color=adjmat.columns.values, size=size, edge_size=size, edge_color='#000FFF', cmap='Set1')
d3.set_node_properties(color=adjmat.columns.values,edge_color='#000FFF', cmap='Set1')
d3.show(filepath="../" + publishers_type + ".html", title = publishers_type)